In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationBufferMemory(
    return_messages=True,
)


examples = [
    {
        "question": "Life of Pi",
        "answer": "👳‍🚣‍🐅",
    },
    {
        "question": "Tangled",
        "answer": "👸🧙‍💇",
    },
    {
        "question": "Charlie and the Chocolate Factory",
        "answer": "👦🍫🏭",
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert, you must have to response the answer with three emojis.",
        ),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)

In [22]:
invoke_chain("Transformer")

🚗🤖💥


In [23]:
invoke_chain("Top Gun")

✈️🔥🕶️


In [24]:
invoke_chain("What was the movie I asked first?")

🤖🚗💥


In [25]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Transformer'),
  AIMessage(content='🚗🤖💥'),
  HumanMessage(content='Top Gun'),
  AIMessage(content='✈️🔥🕶️'),
  HumanMessage(content='What was the movie I asked first?'),
  AIMessage(content='🤖🚗💥')]}